# 电子战对抗仿真系统演示

本笔记本演示电子战对抗仿真系统的基本功能。

In [1]:
# 导入必要的库
import sys
import os

# 添加项目路径
sys.path.append('..')

from src.core.patterns.strategy import ScenarioFactory
from src.core.factory import EntityFactory
from src.visualization.geoviz import EWVisualizer
import holoviews as hv
hv.extension('bokeh')

TypeError: non-default argument 'radar_params' follows default argument

## 1. 创建一对一对抗想定

In [ ]:
# 创建想定
scenario = ScenarioFactory.create_scenario('one_vs_one')

# 配置参数
config = {
    'radar': {
        'id': 'radar_001',
        'name': '预警雷达',
        'lat': 39.9,
        'lon': 116.4,
        'alt': 50.0,
        'frequency': 3.0,
        'power': 100.0
    },
    'jammer': {
        'id': 'jammer_001',
        'name': '远距干扰机',
        'lat': 40.0,
        'lon': 116.5,
        'alt': 10000.0,
        'power': 1000.0
    }
}

# 设置想定
scenario.setup(config)

print(f'想定创建完成: {scenario.name}')
print(f'雷达: {scenario.radars[0].name}')
print(f'干扰机: {scenario.jammers[0].name}')

## 2. 执行仿真

In [ ]:
# 执行仿真
results = scenario.execute()

print('仿真结果:')
print(f"干扰是否有效: {results['result']['effective']}")
print(f"干信比: {results['result']['j_s_ratio']:.1f} dB")
print(f"探测概率: {results['result']['detection_probability']*100:.1f}%")
print(f"传播损耗: {results['result']['propagation_loss']:.1f} dB")
print(f"距离: {results['result']['distance_km']:.1f} km")

## 3. 效能评估

In [ ]:
# 评估结果
assessment = scenario.assess()

print('效能评估:')
print(f"干扰成功率: {assessment['jam_success_rate']:.1f}%")
print(f"建议战术: {', '.join(assessment['suggested_tactics'])}")

## 4. 可视化

In [ ]:
# 创建可视化
ew_map = EWVisualizer.create_coverage_map(scenario.radars, scenario.jammers)

# 显示地图
ew_map

## 5. 信号分析图表

In [ ]:
# 创建信号分析图表
import numpy as np

# 生成示例数据
simulation_results = {
    'j_s_ratio_history': np.random.randn(100).cumsum() + 10,
    'detection_probability': {
        '雷达1': 0.8,
        '雷达2': 0.6,
        '雷达3': 0.9
    },
    'propagation_loss': 120.5
}

# 创建分析图表
analysis_plot = EWVisualizer.create_signal_analysis_plot(simulation_results)

if analysis_plot:
    analysis_plot

## 6. 频谱分析

In [ ]:
# 创建频谱分析
frequencies = np.linspace(100, 10000, 1000)
powers = -50 + 10 * np.sin(frequencies/1000) + np.random.randn(1000)*5

radar_freqs = [3000, 5000, 8000]
jammer_freqs = [2500, 3000, 3500]

spectrum_plot = EWVisualizer.create_spectrum_analysis(
    frequencies, powers, radar_freqs, jammer_freqs
)

spectrum_plot

## 7. 多对多对抗示例

In [ ]:
# 创建多对多想定
many_scenario = ScenarioFactory.create_scenario('many_vs_many')

# 配置网络对抗
network_config = {
    'radar_network': [
        {
            'id': 'net_radar_1',
            'name': '网络雷达1',
            'lat': 39.9,
            'lon': 116.4,
            'frequency': 3.0,
            'power': 100.0
        },
        {
            'id': 'net_radar_2',
            'name': '网络雷达2',
            'lat': 40.0,
            'lon': 116.5,
            'frequency': 2.5,
            'power': 80.0
        }
    ],
    'jammer_network': [
        {
            'id': 'net_jammer_1',
            'name': '网络干扰机1',
            'lat': 40.1,
            'lon': 116.6,
            'power': 1200.0
        },
        {
            'id': 'net_jammer_2',
            'name': '网络干扰机2',
            'lat': 40.2,
            'lon': 116.7,
            'power': 1500.0
        }
    ]
}

many_scenario.setup(network_config)

print(f'雷达网规模: {len(many_scenario.radars)}')
print(f'干扰网规模: {len(many_scenario.jammers)}')

# 网络对抗可视化
network_map = EWVisualizer.create_coverage_map(
    many_scenario.radars, 
    many_scenario.jammers
)

network_map